## This notebook will help you train a vanilla Point-Cloud AE with the basic architecture we used in our paper.
    (it assumes latent_3d_points is in the PYTHONPATH and the structural losses have been compiled)

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "/home/gy46/")

In [2]:
import sys
sys.path.insert(0, "/home/gy46/")

In [3]:
import os.path as osp

from latent_3d_points.src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from latent_3d_points.src.autoencoder import Configuration as Conf
from latent_3d_points.src.point_net_ae import PointNetAutoEncoder

from latent_3d_points.src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder

from latent_3d_points.src.tf_utils import reset_tf_graph
from latent_3d_points.src.general_utils import plot_3d_point_cloud

Define Basic Parameters

In [4]:
top_out_dir = '../data/'                      # Use to save Neural-Net check-points etc.
top_in_dir = '../data/ShapeNetCore.v2.PC15k/' # Top-dir of where point-clouds are stored.

# experiment_name = 'single_class_ae_emd'
n_pc_points = 2048                # Number of points per model.
bneck_size = 128                  # Bottleneck-AE size
# ae_loss = 'chamfer'                   # Loss to optimize: 'emd' or 'chamfer'
ae_loss = 'emd'                   # Loss to optimize: 'emd' or 'chamfer'
# class_name = raw_input('Give me the class name (e.g. "chair"): ').lower()
# class_name = 'airplane'
class_name = 'chair'
experiment_name = 'shapenetcorev2_%s_ae_%s'%(ae_loss, class_name)
print(experiment_name)

shapenetcorev2_emd_ae_chair


Load Point-Clouds

In [5]:
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id, 'train')
print(syn_id)
print(class_dir)

03001627
../data/ShapeNetCore.v2.PC15k/03001627/train


In [6]:
all_pc_data = load_all_point_clouds_under_folder(
    class_dir, n_threads=8, file_ending='.npy', max_num_points=2048, verbose=True, normalize=True)

4612 pclouds were loaded. They belong in 1 shape-classes.


Load default training parameters (some of which are listed beloq). For more details please print the configuration object.

    'batch_size': 50   
    
    'denoising': False     (# by default AE is not denoising)

    'learning_rate': 0.0005

    'z_rotate': False      (# randomly rotate models of each batch)
    
    'loss_display_step': 1 (# display loss at end of these many epochs)
    'saver_step': 10       (# over how many epochs to save neural-network)

In [7]:
train_params = default_train_params()

In [8]:
encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)
train_dir = create_dir(osp.join(top_out_dir, experiment_name))

In [9]:
conf = Conf(n_input = [n_pc_points, 3],
            loss = ae_loss,
            training_epochs = train_params['training_epochs'],
            batch_size = train_params['batch_size'],
            denoising = train_params['denoising'],
            learning_rate = train_params['learning_rate'],
            train_dir = train_dir,
            loss_display_step = train_params['loss_display_step'],
            saver_step = train_params['saver_step'],
            z_rotate = train_params['z_rotate'],
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
conf.experiment_name = experiment_name
conf.held_out_step = 5   # How often to evaluate/print out loss on 
                         # held_out data (if they are provided in ae.train() ).
conf.save(osp.join(train_dir, 'configuration'))

If you ran the above lines, you can reload a saved model like this:

In [10]:
load_pre_trained_ae = False
restore_epoch = 1000
if load_pre_trained_ae:
    conf = Conf.load(train_dir + '/configuration')
    reset_tf_graph()
    ae = PointNetAutoEncoder(conf.experiment_name, conf)
    ae.restore_model(conf.train_dir, epoch=restore_epoch)

Build AE Model.

In [11]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

Building Encoder
('encoder_conv_layer_0', 'conv params = ', 256)
('bnorm params = ', 128)
Tensor("shapenetcorev2_emd_ae_chair_2/Relu:0", shape=(?, 2048, 64), dtype=float32)
('output size:', 131072, '\n')
('encoder_conv_layer_1', 'conv params = ', 8320)
('bnorm params = ', 256)
Tensor("shapenetcorev2_emd_ae_chair_2/Relu_1:0", shape=(?, 2048, 128), dtype=float32)
('output size:', 262144, '\n')
('encoder_conv_layer_2', 'conv params = ', 16512)
('bnorm params = ', 256)
Tensor("shapenetcorev2_emd_ae_chair_2/Relu_2:0", shape=(?, 2048, 128), dtype=float32)
('output size:', 262144, '\n')
('encoder_conv_layer_3', 'conv params = ', 33024)
('bnorm params = ', 512)
Tensor("shapenetcorev2_emd_ae_chair_2/Relu_3:0", shape=(?, 2048, 256), dtype=float32)
('output size:', 524288, '\n')
('encoder_conv_layer_4', 'conv params = ', 32896)
('bnorm params = ', 256)
Tensor("shapenetcorev2_emd_ae_chair_2/Relu_4:0", shape=(?, 2048, 128), dtype=float32)
('output size:', 262144, '\n')
Tensor("shapenetcorev2_emd_ae

Train the AE (save output to train_stats.txt) 

In [ ]:
buf_size = 1 # Make 'training_stats' file to flush each output line regarding training.
fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
train_stats = ae.train(all_pc_data, conf, log_file=fout)
fout.close()

('Epoch:', '0001', 'training time (minutes)=', '0.2526', 'loss=', '0.495435009')
INFO:tensorflow:../data/shapenetcorev2_emd_ae_chair/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0002', 'training time (minutes)=', '0.2505', 'loss=', '0.371383400')
('Epoch:', '0003', 'training time (minutes)=', '0.2513', 'loss=', '0.346661607')
('Epoch:', '0004', 'training time (minutes)=', '0.2541', 'loss=', '0.331389680')
('Epoch:', '0005', 'training time (minutes)=', '0.2586', 'loss=', '0.323597168')
('Epoch:', '0006', 'training time (minutes)=', '0.2574', 'loss=', '0.317598392')
('Epoch:', '0007', 'training time (minutes)=', '0.2575', 'loss=', '0.310558008')
('Epoch:', '0008', 'training time (minutes)=', '0.2586', 'loss=', '0.304355998')
('Epoch:', '0009', 'training time (minutes)=', '0.2575', 'loss=', '0.303986461')
('Epoch:', '0010', 'training time (minutes)=', '0.2566', 'loss=', '0.301677753')
INFO:tensorflow:../data/shapenetcorev2_emd_ae_chair/models.ckpt-10

('Epoch:', '0089', 'training time (minutes)=', '0.2557', 'loss=', '0.245389926')
('Epoch:', '0090', 'training time (minutes)=', '0.2593', 'loss=', '0.245260024')
INFO:tensorflow:../data/shapenetcorev2_emd_ae_chair/models.ckpt-90 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0091', 'training time (minutes)=', '0.2539', 'loss=', '0.245812745')
('Epoch:', '0092', 'training time (minutes)=', '0.2548', 'loss=', '0.246069302')
('Epoch:', '0093', 'training time (minutes)=', '0.2565', 'loss=', '0.244949653')
('Epoch:', '0094', 'training time (minutes)=', '0.2576', 'loss=', '0.244025043')
('Epoch:', '0095', 'training time (minutes)=', '0.2543', 'loss=', '0.245388958')
('Epoch:', '0096', 'training time (minutes)=', '0.2594', 'loss=', '0.244997187')
('Epoch:', '0097', 'training time (minutes)=', '0.2594', 'loss=', '0.244544880')
('Epoch:', '0098', 'training time (minutes)=', '0.2603', 'loss=', '0.244159862')
('Epoch:', '0099', 'training time (minutes)=', '0.2546', 'loss=',

Get a batch of reconstuctions and their latent-codes.

In [ ]:
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id, 'val')
print(syn_id)
print(class_dir)
all_pc_data = load_all_point_clouds_under_folder(
    class_dir, n_threads=8, file_ending='.npy', max_num_points=2048, verbose=True, normalize=True)

In [ ]:
feed_pc, feed_model_names, _ = all_pc_data.next_batch(10)
reconstructions = ae.reconstruct(feed_pc)[0]
latent_codes = ae.transform(feed_pc)

Use any plotting mechanism such as matplotlib to visualize the results.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
def plot_point_cloud(xs, ys, zs, s=1):
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(xs, ys, zs, s=s)
    plt.show()


In [ ]:
import random
# i = random.choice(range(reconstructions.shape[0]))
for i in range(reconstructions.shape[0]):
    print("Recon")
    plot_point_cloud(reconstructions[i][:, 0], reconstructions[i][:, 2], 
                     reconstructions[i][:, 1]);
    print("Gtrs")
    plot_point_cloud(feed_pc[i][:, 0], feed_pc[i][:, 2], feed_pc[i][:, 1]);


# (Deprecated) Generate the latent codes for train/test/val sets

In [ ]:
# Data Loader that takes a list of file names
import os
import numpy as np
from tqdm import tqdm as tqdm
from torch.utils.data import Dataset, DataLoader
from latent_3d_points.src.in_out import snc_synth_id_to_category
cate_to_synsetid = {v:k for k,v in snc_synth_id_to_category.items()}

class Uniform15KPC(Dataset):

    def __init__(self, root_dir, subdirs, sample_size=2048, split='train'):
        self.root_dir = root_dir
        self.split = split
        self.sample_size = sample_size
        self.subdirs = subdirs

        self.stats = []
        self.train_points = []
        self.test_points  = []
        for subd in tqdm(self.subdirs, desc='Subdirectories', leave=True):
            sub_path = os.path.join(root_dir, subd, self.split)
            for x in tqdm(os.listdir(sub_path), desc='shapes', leave=False):
                if not x.endswith('.npy'):
                    continue

                obj_fname = os.path.join(sub_path, x)
                try:
                    point_cloud = np.load(obj_fname) # (15k, 3)
                except Exception as e:
                    print("Exception encountered loading :%s"%obj_fname)
                    print(e)
                    continue
                assert point_cloud.shape[0] == 15000
                tr_pc = point_cloud[:10000, [0,2,1]]
                te_pc = point_cloud[10000:, [0,2,1]]
                m = point_cloud.reshape(15000,3).mean(axis=0).reshape(1,1,3)
                s = point_cloud.reshape(15000*3).std().reshape(1,1,1)
                self.stats.append((m, s))
                
                self.train_points.append((tr_pc[np.newaxis,...] - m)/s)
                self.test_points.append((te_pc[np.newaxis,...] - m)/s)
                

        self.tr_sample_size = min(10000, sample_size)
        self.te_sample_size = min(5000, sample_size)
        print("Total number of data:%d"%len(self.train_points))
        print("Min number of points: (train)%d (test)%d"\
              %(self.tr_sample_size, self.te_sample_size))
        
        

    def __len__(self):
        return len(self.train_points)

    def __getitem__(self, idx):
        tr_out = self.train_points[idx]
        tr_idxs = np.random.choice(tr_out.shape[1], self.tr_sample_size)
        tr_out = tr_out[0,tr_idxs,:]

        te_out = self.test_points[idx]
        te_idxs = np.random.choice(te_out.shape[1], self.te_sample_size)
        te_out = te_out[0,te_idxs,:]

        return tr_out, te_out

    

class ShapeNet15kPointClouds(Uniform15KPC):

    def __init__(self, root_dir="../data/ShapeNetV1PCOutput",
                 categories=['airplane'], sample_size=2048, split='train'):
        self.root_dir = root_dir
        self.split = split
        assert self.split in ['train', 'test', 'val']
        self.sample_size = sample_size
        self.cates = categories
        self.synset_ids = [cate_to_synsetid[c] for c in self.cates]
        super(ShapeNet15kPointClouds, self).__init__(root_dir, self.synset_ids, sample_size=sample_size, split=split)

batch_size=128
tr_ds = ShapeNet15kPointClouds(categories=[class_name], split='train')
tr_loader = DataLoader(dataset=tr_ds, batch_size=batch_size,
                       shuffle=False, num_workers=4, drop_last=False)
print(len(tr_loader))
val_ds = ShapeNet15kPointClouds(categories=[class_name], split='val')
val_loader = DataLoader(dataset=val_ds, batch_size=batch_size,
                        shuffle=False, num_workers=4, drop_last=False)
print(len(val_loader))
te_ds = ShapeNet15kPointClouds(categories=[class_name], split='test')
te_loader = DataLoader(dataset=te_ds, batch_size=batch_size,
                       shuffle=False, num_workers=4, drop_last=False)
print(len(te_loader))

In [ ]:
def save(loader, split):
    all_tr_latents = []
    all_te_latents = []
    all_tr = []
    all_te = []
    for tr_pc, te_pc in tqdm(loader):
        tr_pc_latent = ae.transform(tr_pc)
        te_pc_latent = ae.transform(te_pc)
        all_tr_latents.append(tr_pc_latent)
        all_te_latents.append(te_pc_latent)
        all_tr.append(tr_pc)
        all_te.append(te_pc)

    print(len(all_tr_latents), len(all_te_latents))
    all_tr_latents = np.concatenate(all_tr_latents)
    all_te_latents = np.concatenate(all_te_latents)
    all_tr = np.concatenate(all_tr)
    all_te = np.concatenate(all_te)
    print(all_tr_latents.shape)
    print(all_te_latents.shape)
    tr_latent_save_dir = "../data/ShapeNetV1PCOutput_latent/%s/%s_%s_latent_tr.npy"\
                         %(cate_to_synsetid[class_name], split, ae_loss)
    te_latent_save_dir = "../data/ShapeNetV1PCOutput_latent/%s/%s_%s_latent_te.npy"\
                         %(cate_to_synsetid[class_name], split, ae_loss)
    tr_pc_save_dir = "../data/ShapeNetV1PCOutput_latent/%s/%s_%s_pc_tr.npy"\
                     %(cate_to_synsetid[class_name], split, ae_loss)
    te_pc_save_dir = "../data/ShapeNetV1PCOutput_latent/%s/%s_%s_pc_te.npy"\
                     %(cate_to_synsetid[class_name], split, ae_loss)
    print(tr_latent_save_dir)
    print(te_latent_save_dir)
    print(tr_pc_save_dir)
    print(te_pc_save_dir)
    np.save(tr_latent_save_dir, all_tr_latents)
    np.save(te_latent_save_dir, all_te_latents)
    np.save(tr_pc_save_dir, all_tr)
    np.save(te_pc_save_dir, all_te)
    
save(tr_loader, 'train')
save(val_loader, 'val')
save(te_loader, 'test')

# Evaluate the model

In [ ]:
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id, 'val')
print(syn_id)
print(class_dir)
all_pc_data = load_all_point_clouds_under_folder(
    class_dir, n_threads=8, file_ending='.npy', max_num_points=2048, verbose=True, normalize=True)

In [ ]:
feed_pc, _, _ = all_pc_data.full_epoch_data()
feed_pc_tr_all = feed_pc[:, :n_pc_points]
feed_pc_te_all = feed_pc[:, -n_pc_points:]
print(feed_pc_tr_all.shape)
print(feed_pc_te_all.shape)

In [ ]:
all_sample = []
all_ref = []
for i in range(feed_pc_tr_all.shape[0]):
    feed_pc_tr = feed_pc_tr_all[i:i+1]
    feed_pc_te = feed_pc_te_all[i:i+1]
    reconstructions = ae.reconstruct(feed_pc_tr)[0]
    all_sample.append(reconstructions)
    all_ref.append(feed_pc_te)

all_sample = np.concatenate(all_sample)
all_ref = np.concatenate(all_ref)
all_sample.shape, all_ref.shape

In [ ]:
from latent_3d_points.src.evaluation_metrics_fast import MMD_COV_EMD_CD
mmd_emd, mmd_cd, cov_emd, cov_cd = MMD_COV_EMD_CD(all_sample, all_ref, 100, verbose=True)


In [ ]:
print(experiment_name)
print("MMD-EMD:%s"%mmd_emd)
print("MMD-CD:%s"%mmd_cd)
# print("COV-EMD:%s"%cov_emd)
# print("COV-CD:%s"%cov_cd)